# Additional - Markov Chain with Image

Here is a small code snippet for running markov chain with image. Just play around with it.

with the data from Bauhaus paint</br>
[<img src="images/paint.jpg" width="300x"/>](originalPic.jpeg)

[<img src="images/Markov_directional.jpeg" width="250x"/>](Markov_directional.jpeg)
[<img src="images/markovProcess.gif" width="250x"/>](process.gif)


In [1]:
from collections import defaultdict, Counter
from PIL import Image
import numpy as np

import random
class MarkovChain(object):
    def __init__(self,bucket_size = 10,four_neighbour=True, directional=False):
        self.bucket_size = bucket_size
        self.weights = defaultdict(Counter)
        self.four_neighbour = four_neighbour
        self.directional = directional

    def norm(self,_pixel):
        return _pixel // self.bucket_size
    def denorm(self,_pixel):
        return _pixel * self.bucket_size
    
    def get_neighbors(self, x, y):
        if self.four_neighbour:
            return [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]
        else:
            return [(x + 1, y),
                    (x - 1, y),
                    (x, y + 1),
                    (x, y - 1),
                    (x + 1, y + 1),
                    (x - 1, y - 1),
                    (x - 1, y + 1),
                    (x + 1, y - 1)]
    
    def get_neighbors_dir(self, x, y):
        if self.four_neighbour:
            return {'r': (x + 1, y), 'l': (x - 1, y), 'b': (x, y + 1), 't': (x, y - 1)}
        else:
            return dict(zip(['r', 'l', 'b', 't', 'br', 'tl', 'tr', 'bl'],
                            [(x + 1, y),
                             (x - 1, y),
                             (x, y + 1),
                             (x, y - 1),
                             (x + 1, y + 1),
                             (x - 1, y - 1),
                             (x - 1, y + 1),
                             (x + 1, y - 1)]))
    def train(self,img):
        width,height = img.size
        img = np.array(img)[:,:,:3]
        for x in range(height):
            for y in range(width):
                pixel = tuple(self.norm(img[x,y]))
                for neighbor in self.get_neighbors(x,y):
                    try:
                        self.weights[pixel][tuple(self.norm(img[neighbor]))]+=1
                    except IndexError:
                        continue
        self.directional = False
    def train_direction(self, img):
        self.weights = defaultdict(lambda: defaultdict(Counter))
        width, height = img.size
        img = np.array(img)[:, :, :3]
        for x in range(height):
            for y in range(width):
                pixel = tuple(self.norm(img[x, y]))
                for _dir, neighbor in self.get_neighbors_dir(x, y).items():
                    try:
                        self.weights[pixel][_dir][tuple(self.norm(img[neighbor]))] += 1
                    except IndexError:
                        continue
        self.directional = True
        
    def generate(self,initial=None,width=400, height=400):
        
        if initial is None:
            initial = random.choice(list(self.weights.keys()))
        #print(initial)
            
        img = Image.new('RGB',(width,height),(255,255,255))
        img = np.array(img)
        img_out = np.array(img.copy())
        
        start_position = (random.randint(0,height),random.randint(0,width))
        img[start_position] = initial
        BFS = [start_position]
        isFilled = set()
        while BFS:
            x,y = BFS.pop()
            if(x,y) in isFilled:
                continue
            else:
                isFilled.add((x,y))
            
            pixel = img[x,y]
            node = self.weights[tuple(pixel)]
            img_out[x,y] = self.denorm(pixel)
            
            if self.directional:
                keys = {dir: list(node[dir].keys()) for dir in node}
                neighbors = list(self.get_neighbors_dir(x, y).items())
                counts = {dir: np.array(list(node[dir].values()), dtype=np.float32) for dir in keys}
                key_index = {dir: np.arange(len(node[dir])) for dir in keys}
                ps = {dir: counts[dir] / counts[dir].sum() for dir in keys}
            
            #BFS without Direction
            else:
                keys = list(node.keys())
                neighbors = self.get_neighbors(x,y)
                counts = np.array(list(node.values()),dtype=np.float32)
                key_index = np.arange(len(keys))
                ps = counts / counts.sum()
            #print(neighbors)
            np.random.shuffle(neighbors)
            
            for neighbor in neighbors:
                try:
                    if self.directional:
                        direction = neighbor[0]
                        neighbor = neighbor[1]
                        if neighbor not in isFilled:
                            col_idx = np.random.choice(key_index[direction], p=ps[direction])
                            img[neighbor] = keys[direction][col_idx]
                    else:
                        col_idx = np.random.choice(key_index, p=ps)
                        if neighbor not in isFilled:
                            img[neighbor] = keys[col_idx]
                except IndexError:
                    pass
                except ValueError:
                    continue
                if 0 <= neighbor[1] < width and 0 <= neighbor[0] < height:
                    BFS.append(neighbor)
                    
        return Image.fromarray(img_out)


## adding the training picture

In [2]:
image = Image.open("images/paint.jpg")

#bucket the pixel for compressing image RGB
bucket_size = 15

bucketed = Image.fromarray((np.array(image)//bucket_size) * bucket_size)
#bucketed.show()

## Training 

In [3]:
chain = MarkovChain(bucket_size=bucket_size, 
                    four_neighbour=True,
                    directional=False)
chain.train_direction(bucketed) 

## Show the image

In [4]:
output = chain.generate(width=400, height=400)
output.show()